In [1]:
import gym
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from pprint import pprint
import utils
import torch
from torch.utils.tensorboard import SummaryWriter
import os
from collections import deque
import random

envname = 'BipedalWalker-v3'
envname = 'LunarLander-v2'
#envname = 'CartPole-v0'
# envname = 'Acrobot-v1'
env = gym.make(envname)
env

<TimeLimit<LunarLander<LunarLander-v2>>>

In [2]:
class ReplayBuffer():
    def __init__(self, max_len=100000):
        self.old_obs = deque([])
        self.a = deque([])
        self.r = deque([])
        self.obs = deque([])
        self.done = deque([])
        self.size = max_len
        self.length = 0
        self.max_len = max_len
    def __len__(self):
        return self.length
    def remover(self):
        if self.length > self.max_len:
            self.a.popleft()
            self.old_obs.popleft()
            self.r.popleft()
            self.obs.popleft()
            self.done.popleft()
            self.length -= 1
    def add(self, old_obs, a, r, obs, done):
        self.old_obs.append(old_obs)
        self.a.append(a)
        self.r.append(r)
        self.obs.append(obs)
        self.done.append(done)
        self.length += 1
        self.remover()
    def get(self, bsize):
        lidx = random.sample(range(0, self.length), bsize)
        old_obs = [self.old_obs[i] for i in lidx]
        a       = [self.a[i] for i in lidx]
        r       = [self.r[i] for i in lidx]
        obs     = [self.obs[i] for i in lidx]
        done    = [self.done[i] for i in lidx]
        
        old_obs = torch.tensor(list(old_obs), dtype=torch.float).detach()
        a       = torch.tensor(list(a)      , dtype=torch.float).detach()
        r       = torch.tensor(list(r)      , dtype=torch.float).detach().unsqueeze(-1)
        obs     = torch.tensor(list(obs)    , dtype=torch.float).detach()
        done    = torch.tensor(list(done)   , dtype=torch.float).detach().unsqueeze(-1)
        return old_obs, a, r, obs, done
    
repbuff = ReplayBuffer(1000000)

In [4]:
class DQN(nn.Module):
    def __init__(self, idim, hdim, h2dim, odim):
        super().__init__()
        self.lin1 = nn.Linear(idim, hdim)
        self.lin2 = nn.Linear(hdim, h2dim)
        self.lin3 = nn.Linear(h2dim, odim)
        
        self.odim = odim
    def forward(self, x):
        h = self.lin1(x)
        h = F.relu(h)
        h = self.lin2(h)
        h = F.relu(h)
        h = self.lin3(h)
        return h
    def act(self, obs, epsilon=0.1, debug=False):
        obs = torch.tensor(obs).float()
        try:
            obs.shape[1]
        except:
            obs = obs.unsqueeze(0)
        qs = self.forward(obs)
        ii = 0
        if random.uniform(0, 1) > epsilon:
            ii = torch.argmax(qs, dim=-1).item()
            if debug:
                print(ii, "u prvoj", qs.shape)
        else:
            ii = random.randint(0, self.odim-1)
            if debug:
                print(ii, "u drugoj")
        print(ii, "ii")
        return ii
    
print(env.action_space, env.observation_space)
adim = 4 #env.action_space.shape[0]
sdim = env.observation_space.shape[0]
dqn1 = DQN(sdim, 512, 256, adim)
dqn2 = DQN(sdim, 512, 256, adim)

import copy
tdqn1 = copy.deepcopy(dqn1)
tdqn2 = copy.deepcopy(dqn2)

dqn1

Discrete(4) Box(8,)


DQN(
  (lin1): Linear(in_features=8, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=4, bias=True)
)

In [5]:
opt1 = torch.optim.Adam(dqn1.parameters(), lr=1e-3)
opt2 = torch.optim.Adam(dqn2.parameters(), lr=1e-3)

In [8]:
exname = 'dqn_lunar_2layer_reallybig_lr=1e-3_ispravka epsis'
os.system('mkdir tb/' + exname)
writer = SummaryWriter('tb/'+exname)

bsize = 512
warmup = 400 
gamma = 1
frst = torch.tensor(list(range(bsize)), dtype=torch.long).unsqueeze(-1)
loss1 = 0; 
for ep in range(1850, 100000):
    obs = env.reset()
    if ep % 50==0:
        showit()
    if ep%2==0 or bsize*100>len(repbuff):
        rss=0
        for step in range(3000):
            if ep>warmup:
                a = dqn1.act(obs, epsilon=0.2)
            else:
                a = env.action_space.sample()
            oldobs = obs
            obs, r, done, info = env.step(a)
            rss += r
            repbuff.add(oldobs, [a], r, obs, done)
            if done:
                obs = env.reset()
                break
        writer.add_scalar('a/reward', rss, ep)
    if bsize*10 > len(repbuff):
        continue
    oldobs, a, r, obs, done = repbuff.get(bsize)
    for opt_step in range(3):
        target1 = torch.max(tdqn1(obs), dim=-1)[0].detach().unsqueeze(-1) * (1-done)
        target2 = torch.max(tdqn2(obs), dim=-1)[0].detach().unsqueeze(-1) * (1-done)
        target = torch.min(target1, target2)
#         print(target, target1, target2)
        calc1 = dqn1(oldobs)[frst, a.long()]
        calc2 = dqn2(oldobs)[frst, a.long()]
        
        loss1 = ((r + gamma*target - calc1)**2).mean()
        loss2 = ((r + gamma*target - calc2)**2).mean()
        opt1.zero_grad(); opt2.zero_grad(); 
        loss1.backward(); loss2.backward();
        opt1.step(); opt2.step();
    writer.add_scalar('a/dqn1', loss1.item(), ep)
    writer.add_scalar('a/dqn2', loss2.item(), ep)
    
    polyak(dqn1, tdqn1, 1-1/50)
    polyak(dqn2, tdqn2, 1-1/50)
    
    if ep%25==0:
        print(ep, loss1.item())

AttributeError: 'int' object has no attribute 'shape'

In [11]:
len(repbuff)

0

In [5]:
def polyak(a, b, alfa=0.99):
    for namea, parama in a.named_parameters():
        for nameb, paramb in b.named_parameters():
            if namea == nameb:
                paramb.data = paramb.data*alfa + parama.data*(1-alfa)
    return b 

In [7]:
def showit(debug=False):
    dons = 0
    obs = env.reset()
    env.seed(10)
    for t in range(1500):
            env.render()
            a = dqn1.act(obs, epsilon=0.0, debug=debug)
            obs, reward, done, info = env.step(a)   
            #print(dqn(torch.tensor(obs).float()), a)
            
            if done:
#                 print("r"+"="*20, reward, t)
                break
                sf = 0
            time.sleep(0.001)
            print(info)
            print(t, "/200", reward, end='\r')

env.close()
cv2.destroyAllWindows()

In [151]:
env = gym.make(envname)
showit()

{}
{}/200 -1.2330385227876945
{}/200 -1.3286540670061697
{}/200 -1.2987542306269404
{}/200 -1.2685514151842199
{}/200 -1.2361249820407352
{}/200 -1.2015712267811693
{}/200 -1.165034169997739
{}/200 2.336379735116833
{}/200 -0.44788837710208784
{}/200 4.064701865062102
{} /200 -1.0438919041471308
{} /200 2.642787448041145
{} /200 3.17580876347962
{} /200 -0.9910051633102626
{} /200 2.4478605368365836
{} /200 1.5637387152087399
{} /200 0.6996749753927076
{} /200 0.07172915407583674
{} /200 1.5119740329442208
{} /200 0.8393550656925128
{} /200 2.3718253907385476
{} /200 0.01362206574148786
{} /200 2.591059267065231
{} /200 2.575737091461815
{} /200 0.8959632559498629
{} /200 2.7374827185712265
{} /200 0.6880619114591979
{} /200 1.8708736598903328
{} /200 0.10616193439503377
{} /200 2.8882312881963346
{} /200 -0.44169021433526723
{} /200 2.9842305660924753
{} /200 1.6889633492732912
{} /200 -0.3389118661985435
{} /200 4.323550341410805
{} /200 1.7837191970901471
{} /200 1.3271663124261976
